In [19]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/letterboxd-library/diary.csv


In [20]:
diary = pd.read_csv('/kaggle/input/letterboxd-library/diary.csv')
diary.info(10)
diary.dtypes
diary.describe

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1340 entries, 0 to 1339
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Date            1340 non-null   object 
 1   Name            1340 non-null   object 
 2   Year            1339 non-null   float64
 3   Letterboxd URI  1340 non-null   object 
 4   Rating          1320 non-null   float64
 5   Rewatch         265 non-null    object 
 6   Tags            1005 non-null   object 
 7   Watched Date    1340 non-null   object 
dtypes: float64(2), object(6)
memory usage: 83.9+ KB


<bound method NDFrame.describe of             Date                                         Name    Year  \
0     2020-09-30                               Ford v Ferrari  2019.0   
1     2020-12-08                                     Harakiri  1962.0   
2     2020-05-07                                   The Intent  2016.0   
3     2020-05-13                                   John Henry  2020.0   
4     2020-05-16                                    Apollo 18  2011.0   
...          ...                                          ...     ...   
1335  2023-04-26                            Blade Runner 2049  2017.0   
1336  2023-04-28                   Batman: Under the Red Hood  2010.0   
1337  2023-04-29                         The Bourne Supremacy  2004.0   
1338  2023-04-29  Fantastic Beasts: The Secrets of Dumbledore  2022.0   
1339  2023-04-29                                      Twister  1996.0   

              Letterboxd URI  Rating Rewatch    Tags Watched Date  
0     https://boxd.it

In [30]:
diary['Watched Date'] = pd.to_datetime(diary['Watched Date'])
diary['Date'] = pd.to_datetime(diary['Date'])
diary_sort = diary.sort_values(by=['Watched Date'], ascending=False)
diary_sort.head(10)

,Date,Name,Year,Letterboxd URI,Rating,Rewatch,Tags,Watched Date
1339,2023-04-29,Twister,1996.0,https://boxd.it/4bjVwx,3.5,NaN,hulu,2023-04-28
1338,2023-04-29,Fantastic Beasts: The Secrets of Dumbledore,2022.0,https://boxd.it/4b7F3P,2.0,NaN,hbomax,2023-04-27
1337,2023-04-29,The Bourne Supremacy,2004.0,https://boxd.it/4b8yF5,3.5,NaN,hbomax,2023-04-26
1336,2023-04-28,Batman: Under the Red Hood,2010.0,https://boxd.it/4b3Dcn,3.5,NaN,hbomax,2023-04-25
1335,2023-04-26,Blade Runner 2049,2017.0,https://boxd.it/4aAWLj,4.5,NaN,hulu,2023-04-24
1334,2023-04-24,Rush Hour 2,2001.0,https://boxd.it/4a4khV,3.5,Yes,appletv,2023-04-23
1333,2023-04-24,Power Rangers,2017.0,https://boxd.it/49SyrL,3.0,NaN,NaN,2023-04-22
1332,2023-04-22,Star Wars: Episode II - Attack of the Clones,2002.0,https://boxd.it/49sWFP,2.5,Yes,disney+,2023-04-21
1331,2023-04-22,Star Wars: Episode I - The Phantom Menace,1999.0,https://boxd.it/49iAh7,2.5,Yes,disney+,2023-04-20
1330,2023-04-21,Star Wars: The Rise of Skywalker,2019.0,https://boxd.it/49duhZ,2.5,Yes,disney+,2023-04-19


In [31]:
print(diary_sort.dtypes)

Date              datetime64[ns]
Name                      object
Year                     float64
Letterboxd URI            object
Rating                   float64
Rewatch                   object
Tags                      object
Watched Date      datetime64[ns]
dtype: object


In [37]:
grouped_rating = diary.groupby('Rating').size()
grouped_rating = grouped_rating.sort_values(ascending=False)
print(grouped_rating)

Rating
4.0    330
3.5    327
3.0    267
2.5    138
4.5    113
2.0     78
5.0     48
1.5     17
1.0      2
dtype: int64


In [39]:
grouped_tag = diary.groupby('Tags').size()
grouped_tag = grouped_tag.sort_values(ascending=False)
print(grouped_tag)

Tags
hbomax              318
netflix             263
disney+             142
amazonprime         113
hulu                102
youtube              31
showtime             22
vimeo                 3
appletv               2
hulu, showtime        1
imax                  1
amazongprime          1
netflix, top2020      1
peacock               1
shudder               1
top2020               1
top2020, imax         1
HboMax                1
dtype: int64


In [50]:
diary['Tags'] = diary['Tags'].replace('HboMax', 'hbomax')
diary['Tags'] = diary['Tags'].replace('hulu, showtime', 'showtime')
diary['Tags'] = diary['Tags'].replace('amazongprime', 'amazonprime')
diary['Tags'] = diary['Tags'].replace('netflix, top2020', 'netflix')
diary['Tags'] = diary['Tags'].replace('top2020, imax', 'imax')
diary['Tags'] = diary['Tags'].replace('', np.nan)


grouped_tag = diary.groupby('Tags').size()
grouped_tag = grouped_tag.sort_values(ascending=False)
print(grouped_tag)

##at this point after doing some cleaning i exported this csv and used the TMDb api to 
##add the genres to each of the films in my diary by comparing it to title name

Tags
hbomax         319
netflix        264
disney+        142
amazonprime    114
hulu           102
youtube         31
showtime        23
vimeo            3
appletv          2
imax             2
peacock          1
shudder          1
dtype: int64


In [61]:
movie_diary = pd.read_csv('/kaggle/input/movie-diary/movies.csv')
movie_diary.info(10)
movie_diary.dtypes
movie_diary.describe

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1356 entries, 0 to 1355
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Date            1356 non-null   object 
 1   Name            1356 non-null   object 
 2   Year            1355 non-null   float64
 3   Letterboxd URI  1356 non-null   object 
 4   Rating          1336 non-null   float64
 5   Rewatch         269 non-null    object 
 6   Tags            1017 non-null   object 
 7   Watched Date    1356 non-null   object 
 8   genres          1356 non-null   object 
dtypes: float64(2), object(7)
memory usage: 95.5+ KB


<bound method NDFrame.describe of             Date            Name    Year          Letterboxd URI  Rating  \
0     2023-05-14             Air  2023.0  https://boxd.it/4ff8tn     4.0   
1     2023-05-13         Whip It  2009.0  https://boxd.it/4f1iML     3.5   
2     2023-05-12        The Cook  1918.0  https://boxd.it/4eN8Qv     3.0   
3     2023-05-12         Tangled  2010.0  https://boxd.it/4eM1QL     3.5   
4     2023-05-10  Beau Is Afraid  2023.0  https://boxd.it/4eoySn     3.5   
...          ...             ...     ...                     ...     ...   
1351  2020-05-16       Apollo 18  2011.0  https://boxd.it/19cC6J     1.5   
1352  2020-05-13      John Henry  2020.0  https://boxd.it/18RoWJ     2.0   
1353  2020-05-07      The Intent  2016.0  https://boxd.it/18avRv     1.0   
1354  2020-12-08        Harakiri  1962.0  https://boxd.it/1v6C1b     5.0   
1355  2020-09-30  Ford v Ferrari  2019.0  https://boxd.it/1nxHk5     4.0   

     Rewatch         Tags Watched Date  \
0        Na

In [62]:
movie_diary.head()

,Date,Name,Year,Letterboxd URI,Rating,Rewatch,Tags,Watched Date,genres
0,2023-05-14,Air,2023.0,https://boxd.it/4ff8tn,4.0,NaN,amazonprime,2023-05-13,"['Drama', 'History']"
1,2023-05-13,Whip It,2009.0,https://boxd.it/4f1iML,3.5,NaN,hbomax,2023-05-12,['Drama']
2,2023-05-12,The Cook,1918.0,https://boxd.it/4eN8Qv,3.0,NaN,youtube,2023-05-11,"['Comedy', 'Horror']"
3,2023-05-12,Tangled,2010.0,https://boxd.it/4eM1QL,3.5,NaN,disney+,2023-05-11,"['Animation', 'Family']"
4,2023-05-10,Beau Is Afraid,2023.0,https://boxd.it/4eoySn,3.5,NaN,imax,2023-05-10,"['Comedy', 'Adventure', 'Fantasy']"


In [66]:
grouped_genre = movie_diary.groupby('genres').size()
grouped_genre = grouped_genre.sort_values(ascending=False)
print(grouped_genre)

genres
['Documentary']                                 61
[]                                              48
['Drama']                                       47
['Comedy']                                      40
['Action', 'Adventure', 'Science Fiction']      27
                                                ..
['Comedy', 'Action', 'Family', 'Drama']          1
['Comedy', 'Action', 'Crime']                    1
['Comedy', 'Action', 'Adventure', 'Fantasy']     1
['Animation', 'Thriller']                        1
['Crime', 'Action', 'Comedy', 'Thriller']        1
Length: 538, dtype: int64


In [73]:
movie_ex = movie_diary['genres'].apply(pd.Series)
movie_d2 = movie_diary

movie_d2 = pd.concat([movie_d2, movie_ex], axis=1).drop('genres',axis=1)
print(movie_d2)

            Date            Name    Year          Letterboxd URI  Rating  \
0     2023-05-14             Air  2023.0  https://boxd.it/4ff8tn     4.0   
1     2023-05-13         Whip It  2009.0  https://boxd.it/4f1iML     3.5   
2     2023-05-12        The Cook  1918.0  https://boxd.it/4eN8Qv     3.0   
3     2023-05-12         Tangled  2010.0  https://boxd.it/4eM1QL     3.5   
4     2023-05-10  Beau Is Afraid  2023.0  https://boxd.it/4eoySn     3.5   
...          ...             ...     ...                     ...     ...   
1351  2020-05-16       Apollo 18  2011.0  https://boxd.it/19cC6J     1.5   
1352  2020-05-13      John Henry  2020.0  https://boxd.it/18RoWJ     2.0   
1353  2020-05-07      The Intent  2016.0  https://boxd.it/18avRv     1.0   
1354  2020-12-08        Harakiri  1962.0  https://boxd.it/1v6C1b     5.0   
1355  2020-09-30  Ford v Ferrari  2019.0  https://boxd.it/1nxHk5     4.0   

     Rewatch         Tags Watched Date  \
0        NaN  amazonprime   2023-05-13   
1  

In [74]:
genre_counts = movie_diary['genres'].explode().value_counts()
print(genre_counts)

['Documentary']                                61
[]                                             48
['Drama']                                      47
['Comedy']                                     40
['Action', 'Adventure', 'Science Fiction']     27
                                               ..
['Action', 'Adventure', 'History', 'Drama']     1
['Western', 'Action', 'Thriller']               1
['Mystery', 'Animation', 'Fantasy']             1
['Drama', 'Action', 'Thriller']                 1
['Drama', 'Fantasy', 'Mystery', 'Thriller']     1
Name: genres, Length: 538, dtype: int64
